In [2]:
#importing dependencies and setting app token
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date, datetime
import time
import requests
import sqlite3
from api_keys import MyAppToken
import json
import quandl

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
daynum = today.strftime("%d")
month = today.strftime("%m")
year = today.strftime("%Y")
day = int(daynum) - 7
print(f"Today: {today} \n\tyear: {year} \n\tmonth: {month} \n\tday of month: {daynum} ")

Today: 2020-05-03 
	year: 2020 
	month: 05 
	day of month: 03 


In [3]:
# https://www.quandl.com/data/FRED/DGS10-10-Year-Treasury-Constant-Maturity-Rate
# 10-Year Treasury Constant Maturity Rate
dfTreasury = quandl.get("FRED/DGS10", authtoken=MyAppToken)
dfTreasury['source'] = 'Treasury yield'
dfTreasury.head()

,Value,source
Date,,
1962-01-02,4.06,Treasury yield
1962-01-03,4.03,Treasury yield
1962-01-04,3.99,Treasury yield
1962-01-05,4.02,Treasury yield
1962-01-08,4.03,Treasury yield


In [4]:
# https://www.bloomberg.com/quote/ILM3NAVG:IND
# ILM3NAVG:IND is the Bankrate.com US Home Mortgage 30 Year Fixed National Avg
# Very costly to get that data but Quandl is free
# https://www.quandl.com/data/FMAC/30US-30-Year-Fixed-Rate-Mortgage-Average-in-the-United-States
df30yrFRM = quandl.get("FMAC/30US", authtoken=MyAppToken)
df30yrFRM['source'] = 'Freddie Mac - 30yr FRM avg.'
df30yrFRM.head()

,Value,source
Date,,
1971-04-02,7.33,Freddie Mac - 30yr FRM avg.
1971-04-09,7.31,Freddie Mac - 30yr FRM avg.
1971-04-16,7.31,Freddie Mac - 30yr FRM avg.
1971-04-23,7.31,Freddie Mac - 30yr FRM avg.
1971-04-30,7.29,Freddie Mac - 30yr FRM avg.


In [7]:
dfCombined = pd.merge(
    dfTreasury
    , df30yrFRM
    , on="Date"
    , how="left" # Treasury data goes back to 1961 but 30yr only 1971
    , suffixes=('_Treas', '_30yr')
)

dfCombined = dfCombined.drop(columns=['source_Treas', 'source_30yr'], axis=1)
dfCombined.columns = ['TreasuryYieldRate', '30yrFRMAvg']
dfCombined = dfCombined.reset_index(level=['Date'])
dfCombined = dfCombined.dropna()
dfCombined.tail(250)

,Date,TreasuryYieldRate,30yrFRMAvg
13379,2015-07-30,2.28,3.98
13384,2015-08-06,2.23,3.91
13389,2015-08-13,2.19,3.94
13394,2015-08-20,2.09,3.93
13399,2015-08-27,2.18,3.84
13404,2015-09-03,2.18,3.89
13408,2015-09-10,2.23,3.90
13413,2015-09-17,2.21,3.91
13418,2015-09-24,2.13,3.86
13423,2015-10-01,2.05,3.85


In [11]:
cutoff_start = "June 1, 1998"
dfCombined = dfCombined[dfCombined["Date"] >= cutoff_start].sort_values('Date')
dfCombined.head()

,Date,TreasuryYieldRate,30yrFRMAvg
9087,1998-06-05,5.58,7.05
9092,1998-06-12,5.43,7.04
9097,1998-06-19,5.47,6.94
9102,1998-06-26,5.46,6.96
9111,1998-07-10,5.42,6.91


In [19]:
import plotly.graph_objects as go
np.random.seed(1)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfCombined["Date"], y=dfCombined["TreasuryYieldRate"],
    mode='lines'
    , name='Treasury Yield Rates'
))
fig.add_trace(go.Scatter(x=dfCombined["Date"], y=dfCombined["30yrFRMAvg"],
    mode='lines'
    , name='30yr. Fixed Rate Morgages (Avg)'
))
                                      